<a href="https://colab.research.google.com/github/placeasmedia/text_classification/blob/master/text_as_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import dependencies


In [14]:
!pip install --upgrade torch pytorch_pretrained_bert



Requirement already up-to-date: torch in /usr/local/lib/python3.6/dist-packages (1.5.1+cu101)
     |████████████████████████████████| 133kB 3.0MB/s 


In [16]:
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim

from fastai import *
from fastai.vision import *
from fastai.text import *
from fastai.callbacks import *


# Load data

In [17]:
def load_data(filename):
    df = pd.read_csv(filename)
    selected = ['text']
    non_selected = list(set(df.columns) - set(selected))

    df = df.drop(non_selected, axis=1)
    df = df.dropna(axis=0, how='any', subset=selected)
    df = df.reindex(np.random.permutation(df.index))

#     labels = df[selected[0]].tolist()

#     x_raw= df[selected[0]].apply(lambda x: seg_char(x).split(' ')).tolist() 
    x_raw= df[selected[0]]

    return x_raw

# local data directory

In [29]:
X1 = load_data("https://raw.githubusercontent.com/placeasmedia/text_classification/master/censored.csv")
X2 = load_data("https://raw.githubusercontent.com/placeasmedia/text_classification/master/nonpolitical.csv")
Y1 = np.ones(len(X1), dtype=int)
Y2 = np.zeros(len(X2), dtype=int)
df1 = pd.DataFrame(X1)
df2 = pd.DataFrame(X2)
df = pd.concat([df1, df2])
X = np.array(df)
y = np.concatenate((Y1, Y2))
df['label'] = y

HTTPError: ignored

## Set configuration parameters

In [20]:
#config class
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)

config = Config(
    testing=False,
    bert_model_name="bert-base-chinese", 
    #bert model: Chinese Simplified and Traditional, 12-layer, 768-hidden, 12-heads, 110M parameters
    max_lr=3e-3,
    epochs=1,
    use_fp16=False, #fastai set up ：learner.to_fp16()
    bs=8,
    max_seq_len=128, #text length, can be slow when large
)

# import pytorch_pretrained_bert
from pytorch_pretrained_bert import BertTokenizer
bert_tok = BertTokenizer.from_pretrained(
    config.bert_model_name,
)

# fastai BaseTokenizer
class FastAiBertTokenizer(BaseTokenizer):
    """Wrapper around a BertTokenizer to be a BaseTokenizer in fastai"""
    def __init__(self, tokenizer: BertTokenizer, max_seq_len: int=128, **kwargs):
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __call__(self, *args, **kwargs):
        return self
    # Add token[CLS] and [SEP]
    def tokenizer(self, t:str) -> List[str]:
        """Limits the maximum sequence length"""
        return ["[CLS]"] + self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + ["[SEP]"] 

In [21]:
#最终的tokenizer
fastai_tokenizer = Tokenizer(
    tok_func=FastAiBertTokenizer(bert_tok, max_seq_len=config.max_seq_len), 
    pre_rules=[], 
    post_rules=[]
)

#设置vocab
fastai_bert_vocab = Vocab(list(bert_tok.vocab.keys()))

from sklearn.model_selection import train_test_split

# # Replace NaN value with 0
# df_htl.dropna(how='any', axis=0)

train, val = train_test_split(df)

databunch = TextClasDataBunch.from_df(".", train, val,
                                      tokenizer=fastai_tokenizer,
                                      vocab=fastai_bert_vocab,
                                      include_bos=False,
                                      include_eos=False,
                                      text_cols="text",
                                      label_cols="label",
                                      bs=config.bs,
                                      collate_fn=partial(pad_collate, pad_first=False, pad_idx=0),
                                     )

In [22]:
from pytorch_pretrained_bert.modeling import BertConfig, BertForSequenceClassification

bert_model = BertForSequenceClassification.from_pretrained(config.bert_model_name, num_labels=2)

loss_func = nn.CrossEntropyLoss()

learner = Learner(databunch, bert_model,loss_func=loss_func,metrics=accuracy)


100%|██████████| 382072689/382072689 [00:07<00:00, 51192158.59B/s]


In [28]:
#用样例测试下
learner.predict("刘晓波是无辜的。它共产党才不管你对它有恩有仇呐，只要对它有用，它就宣传。一切都是政治。")

/content
